<a href="https://colab.research.google.com/github/Shafiullah-Shinwari/Shafiullah-Shinwari/blob/main/The_BlindTuner_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from timm import create_model
import numpy as np

# Dummy encryption/decryption functions (replace with actual FHE)
def encrypt_data(X, Y, public_key):
    # Simulate encryption (replace with actual encryption)
    return X, Y  # Placeholder for encrypted data

def decrypt_data(logits, private_key):
    # Simulate decryption (replace with actual decryption)
    return logits  # Placeholder for decrypted logits

def nag_update(W, V, grad, alpha, batch_size, gamma):
    """
    Perform a Nesterov Accelerated Gradient update.
    """
    V_next = (1 - gamma) * W + gamma * V
    W_next = V_next - (alpha / batch_size) * grad
    return W_next, V_next

def train_step(model, cloud_data, optimizer, alpha, batch_size, gamma, public_key, private_key):
    """
    A single training step that mimics the BlindTuner process.
    """
    model.train()
    V_dict = {name: torch.zeros_like(param) for name, param in model.state_dict().items()}

    for Xct, Yct in cloud_data:
        Xct_enc, Yct_enc = encrypt_data(Xct, Yct, public_key)
        logits = model(Xct_enc)
        loss = F.cross_entropy(logits, Yct_enc)
        optimizer.zero_grad()
        loss.backward()

        # Update weights with NAG (using model.named_parameters())
        for name, param in model.named_parameters():
            if param.grad is not None:
                grad = param.grad
                V = V_dict[name]
                param.data, V_dict[name] = nag_update(W=param.data, V=V, grad=grad, alpha=alpha, batch_size=batch_size, gamma=gamma)

        logits_dec = decrypt_data(logits, private_key)
        return logits_dec

def validate_step(model, val_data, Yct_val, public_key, private_key):
    """
    Simulate validation step where the logits are decrypted and loss is computed.
    """
    model.eval()
    val_logits = model(val_data)

    # Calculate validation loss
    val_loss = F.cross_entropy(val_logits, Yct_val)
    return val_loss

def blind_tuner_system(Xtrain, Ytrain, Xval, Yval, public_key, private_key, alpha, batch_size, gamma, epochs):
    # Step 1: Feature Extraction using DEiT
    model = create_model('deit_base_distilled_patch16_224', pretrained=True)

    # Step 2: Simulate encryption
    encrypted_Xtrain, encrypted_Ytrain = encrypt_data(Xtrain, Ytrain, public_key)
    encrypted_Xval, encrypted_Yval = encrypt_data(Xval, Yval, public_key)

    # Prepare DataLoader
    train_loader = torch.utils.data.DataLoader(list(zip(encrypted_Xtrain, encrypted_Ytrain)), batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(list(zip(encrypted_Xval, encrypted_Yval)), batch_size=batch_size, shuffle=False)

    # Step 3: Initialize optimizer
    optimizer = optim.SGD(model.parameters(), lr=alpha, momentum=0.9)

    # Training Loop
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        train_step(model, train_loader, optimizer, alpha, batch_size, gamma, public_key, private_key)

        # Pass encrypted Yval to validate_step
        val_loss = validate_step(model, encrypted_Xval, encrypted_Yval, public_key, private_key)
        print(f"Validation Loss: {val_loss.item()}")

    # Final model transmission (Step 4)
    # Decrypt the model weights to obtain the final model
    Wpt = decrypt_data(model.state_dict(), private_key)
    return Wpt

# Example usage:
if __name__ == "__main__":
    # Example dataset (replace with actual dataset and keys)
    Xtrain, Ytrain = torch.randn(100, 3, 224, 224), torch.randint(0, 1000, (100,))
    Xval, Yval = torch.randn(20, 3, 224, 224), torch.randint(0, 1000, (20,))

    # Placeholder encryption keys (you'll need actual keys from an encryption library)
    public_key = None
    private_key = None

    # Hyperparameters
    alpha = 0.01
    batch_size = 32
    gamma = 0.9
    epochs = 10

    # Run BlindTuner system
    final_model = blind_tuner_system(Xtrain, Ytrain, Xval, Yval, public_key, private_key, alpha, batch_size, gamma, epochs)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Epoch 1/10
Validation Loss: 6.897387504577637
Epoch 2/10
Validation Loss: 6.906926155090332
Epoch 3/10
Validation Loss: 6.9076714515686035
Epoch 4/10
Validation Loss: 6.9077467918396
Epoch 5/10
Validation Loss: 6.907755374908447
Epoch 6/10
Validation Loss: 6.907753944396973
Epoch 7/10
Validation Loss: 6.907753944396973
Epoch 8/10
Validation Loss: 6.907753944396973
Epoch 9/10
Validation Loss: 6.907755374908447
Epoch 10/10
Validation Loss: 6.907755374908447
